In [1]:
from pyairtable import Api
from pyairtable.formulas import match
import pandas as pd
import requests
import json
from collections import defaultdict
from turfpy.transformation import union
from geojson import FeatureCollection, Feature

api_airtable_key = "patfZxFRQiBjvIvvo.677f2eac18aacda776f61eace4bd46c28c6c393db89afd086a41933bd2f39ad2"
MAPBOX_TOKEN = "pk.eyJ1IjoiYXJ0aHVyY2hpcXVldCIsImEiOiJjbHBoaHB6MncwMXNlMmxucDBlYndpaW93In0.Muo3cgXaUJjMC14H9bXfUw"

In [2]:
colonnes_fiches = [
    "codecommune",
    "nomcommune",
    "sectioncadastrale",
    "numeroplan",
    "lieuxdit",
    "nomproprietaire",
    "modedefairevaloir",
    "nomcepage",
    "nomportegreffe",
    "anneeplantation",
    "surfaceplantee"
]

In [3]:
def formater_fiche(df):
    df = (
        df.groupby(["codecommune", "sectioncadastrale", "numeroplan"])
        .agg(
            {
                "nomproprietaire": "first",
                "nomcommune": "first",
                "lieuxdit": "first",
                "nomcepage": list,
                "anneeplantation": list,
                "surfaceplantee": "sum",
            }
        )
        .reset_index()
    )

    df["codecommune"]=df["codecommune"].astype(str).map(lambda x: x[:2]+x[3:])

    df["sectioncadastrale"] = (
        df["sectioncadastrale"]
        .map(lambda x: x.replace(" ", ""))
        .map(lambda x: (2 - len(x)) * "0" + x)
    )

    df["numeroplan"] = (
        df["numeroplan"].astype(str).map(lambda x: (4 - len(x)) * "0" + x)
    )

    df["nomcepage"] = df["nomcepage"].map(lambda x: " & ".join(sorted(set(map(str, x)))))
    df["anneeplantation"] = df["anneeplantation"].map(lambda x: " & ".join(sorted(set(map(str, x)))))
    df["id"] = df["codecommune"] + "000" + df["sectioncadastrale"] + df["numeroplan"]
    return df[["id", "codecommune", "lieuxdit", "nomcepage", "anneeplantation", "surfaceplantee"]]

In [4]:
# def get_features(df, nom):
#   features = []
#   for i in range(df.shape[0]):

#       try:
#           res = requests.get(
#               f"https://apicarto.ign.fr/api/cadastre/parcelle?code_insee={df['codecommune'][i]}&section={df['sectioncadastrale'][i]}&numero={df['numeroplan'][i]}&source_ign=PCI"
#           )
#           res.raise_for_status()
#           coordinates = res.json()["features"][0]["geometry"]["coordinates"][0]
#       except requests.ConnectionError:
#           break
#       except requests.RequestException:
#           break
#       except IndexError:
#           coordinates = [[]]
#       except KeyError:
#           coordinates = [[]]
#       features.append({
#               "id" : abs(hash(df["id"][i])),
#               "type": "Feature",
#               "geometry": {
#                   "type": "Polygon",
#                   "coordinates": coordinates,
#               },
#               "properties": {
#                   "nomproprietaire": nom,
#                   "codecommune": df["codecommune"][i],
#                   "nom": df["lieuxdit"][i],
#                   "cepage": df["nomcepage"][i],
#                   "annee": df["anneeplantation"][i],
#                   "surface_p": int(df["surfaceplantee"][i])
#               }
#           })
#       print(f"{i+1}/{df.shape[0]} parcelle(s) téléchargée(s)")
#   return features

In [5]:
def df_to_dict(df):
    # Grouper le DataFrame par 'codecommune'
    result = df.groupby("codecommune").apply(
        lambda group: group[["id", "codecommune", "lieuxdit", "nomcepage", "anneeplantation", "surfaceplantee"]].to_dict(orient="records")
    ).to_dict()

    return result

In [6]:
api = Api(api_airtable_key)
table = api.table('app08SGfxSPaGDDEH', 'tbl0Iz3svkG3iY4Xo')
records = table.all(formula=match({"Statut": "Pending"}))

In [7]:
# record = records[0]
# csv_url = record['fields']['Parcellaire'][0]['url']

# # Lecture et nettoyage du fichier CSV
# df = pd.read_csv(csv_url, sep=";")
# df.columns = (
#     df.columns.str.replace(" ", "")
#     .str.replace("_", "")
#     .str.replace("-", "")
# )

# # Traitement des données du fichier CSV
# clean_df = formater_fiche(df)
# clean_df[clean_df.codecommune == "51030"].set_index('id').to_dict("index")

In [8]:
record = records[0]

In [9]:
record['fields']

{'Email': 'arthur.chiquet@gmail.com',
 'Statut': 'Pending',
 'Nom': 'Champagne Jacquesson',
 'Adresse': '99 rue du colonel fabien, 51530 dizy',
 'Description': 'sdffsfs\n',
 'Téléphone': '(064) 547-1323',
 'Parcellaire': [{'id': 'attt9XttLsqsxndLq',
   'url': 'https://v5.airtableusercontent.com/v3/u/36/36/1733875200000/tfWWk__zrp7PaI5CMj5X9A/ByECDyOpbOOP-bPlLEeqKiv_HXJdZiZwNGXvcn2l8dc66VqREYcIiWvsfo9hAIiFhKM5nsFGY6XJZ6f4o5iviXvCaxfYBfepK6haVpKjP3X72zIJlebaI1S82cpvQVpxKjQegfg-5fVagJOoBQMV1hYuzGZxVIMhUobpJRVohX93jRpifQf-_9iov4Uv1DYe/4WgWyH2QMkyaQZMIPDwFXn9k9XCmEXcnNlsX00P49BU',
   'filename': 'ParcellaireDetail_29_11_2024.csv',
   'size': 5062,
   'type': 'text/csv'}],
 'Type': 'Viticulture bio',
 'Dernière modification': '2024-12-10T17:34:27.000Z'}

In [10]:
i = 1
# Traitement des enregistrements
for record in records:
    print(f"record {i}/{len(records)}")
    nom = record['fields']['Nom']
    adresse = record['fields']['Adresse']
    csv_url = record['fields']['Parcellaire'][0]['url']
    record['fields'].pop('Parcellaire')
    record['fields'].pop('Statut')

    with open(f'../exploitations/{nom}.json', 'w') as file:
        json.dump(record['fields'], file)



    # Lecture et nettoyage du fichier CSV
    df = pd.read_csv(csv_url, sep=";")
    df.columns = (
        df.columns.str.replace(" ", "")
        .str.replace("_", "")
        .str.replace("-", "")
    )

    # Validation du format des données
    if df.columns.tolist() != colonnes_fiches:
        print("Le format de données ne correspond pas. Veuillez vérifier l'intitulé des colonnes de votre fichier")
        break
    elif df.shape[0] == 0:
        print("Le fichier est vide")
        break

    # Traitement des données du fichier CSV
    clean_df = formater_fiche(df)
    clean_df['nom'] = nom

    for commune in clean_df.codecommune.unique():
        try:
            with open(f'../parcelles/{commune}.json', 'r') as file:
                data = json.load(file)
        except FileNotFoundError:
            data = {}

        filtered_df = clean_df[clean_df.codecommune == commune].set_index('id').to_dict("index")
        updated_data = data.update(filtered_df)
        print(f"Enregistrement du fichier ../parcelles/{commune}.json")
        with open(f'../parcelles/{commune}.json', 'w') as file:
            json.dump(data, file)

    table.update(record['id'], {"Statut": "Available"})
    print("Le fichier JSON des parcelles a été mis à jour avec succès.")

    i+=1

record 1/3
Enregistrement du fichier ../parcelles/51030.json
Enregistrement du fichier ../parcelles/51119.json
Enregistrement du fichier ../parcelles/51210.json
Enregistrement du fichier ../parcelles/51287.json
Le fichier JSON des parcelles a été mis à jour avec succès.
record 2/3
Enregistrement du fichier ../parcelles/51210.json
Le fichier JSON des parcelles a été mis à jour avec succès.
record 3/3
Enregistrement du fichier ../parcelles/51210.json
Le fichier JSON des parcelles a été mis à jour avec succès.
